# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [4]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [5]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [6]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [7]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [9]:
#new
context_user = context.copy()
print(return_CCRMSQL("""the highest paid employee's name""", context_user))

```sql
SELECT name
FROM employees
ORDER BY salary DESC
LIMIT 1;
```


In [10]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("highest-paid employee's name", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query selects the name of the employee who has the highest salary by joining the "employees" table with the "salary" table on the ID_usr field. It then orders the result by salary in descending order and limits the output to only one row, which corresponds to the highest-paid employee.


In [11]:
#new
print(return_CCRMSQL("institution whose graduates have the highest average salary", context_user))

```sql
SELECT institution_name, AVG(salary) AS average_salary
FROM graduates
GROUP BY institution_name
ORDER BY average_salary DESC
LIMIT 1;
```


In [12]:
#old
print(return_CCRMSQL("institution whose graduates have the highest average salary", old_context_user))

This is your SQL:
```sql
SELECT s.Institution, AVG(sa.salary) AS avg_salary
FROM studies s
JOIN employees e ON s.ID_usr = e.ID_usr
JOIN salary sa ON s.ID_usr = sa.ID_usr
GROUP BY s.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```

This SQL query joins the tables `studies`, `employees`, and `salary` based on the user ID. It calculates the average salary for each institution by grouping the results by institution. Then, it orders the results by average salary in descending order and limits the output to show only the institution with the highest average salary.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [13]:
# Test Queries
queries = [
    "highest paid employee's name",
    "institution whose graduates have the highest average salary",
    "employee with the lowest salary and their institution"
]

# Running the experiment with slightly different prompts
for i, query in enumerate(queries):
    print(f"\n--- Query {i+1}: {query} ---")

    # Baseline Prompt
    t0_prompt = [
        {'role': 'system', 'content': """
        You are an SQL expert. Generate a valid SQL query based on the user request.
        The database has the following tables:

        - `employees (ID_usr INT, name VARCHAR)`
        - `salary (ID_usr INT, year DATE, salary FLOAT)`
        - `studies (ID INT, ID_usr INT, Institution VARCHAR, Speciality VARCHAR)`

        Ensure the SQL is efficient and uses correct table joins when necessary.
        Provide only the SQL query in response.
        """}
    ]
    print("t0 Prompt Result:")
    print(return_CCRMSQL(query, t0_prompt))

    # Concise Prompt
    simple_prompt = [
        {'role': 'system', 'content': """
        Database has three tables: employees(ID_usr, name), salary(ID_usr, year, salary), and studies(ID, ID_usr, Institution, Speciality).
        Provide a valid SQLite SQL query for each request.
        """}
    ]
    print("simple Prompt Result:")
    print(return_CCRMSQL(query, simple_prompt))

    # Conversational Prompt
    t1_prompt = [
        {'role': 'system', 'content': """
        You are an expert database assistant. The company has employees with salaries stored in a database. 
        Employees have studied at institutions, and we want to analyze salary trends based on their education.
        Your goal is to generate efficient SQL queries based on user requests.
        """}
    ]
    print("t1 Prompt Result:")
    print(return_CCRMSQL(query, t1_prompt))


--- Query 1: highest paid employee's name ---
t0 Prompt Result:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.salary = (SELECT MAX(salary) FROM salary);
```
simple Prompt Result:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.salary = (SELECT MAX(salary) FROM salary);
```
t1 Prompt Result:
```sql
SELECT name
FROM employees
ORDER BY salary DESC
LIMIT 1;
```

--- Query 2: institution whose graduates have the highest average salary ---
t0 Prompt Result:
```sql
SELECT s.Institution
FROM studies st
JOIN salary s ON st.ID_usr = s.ID_usr
GROUP BY st.Institution
ORDER BY AVG(s.salary) DESC
LIMIT 1;
```
simple Prompt Result:
```sql
SELECT s.Institution, AVG(sa.salary) AS avg_salary
FROM studies s
JOIN salary sa ON s.ID_usr = sa.ID_usr
GROUP BY s.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```
t1 Prompt Result:
```sql
SELECT institution, AVG(salary) AS average_salary
FROM employees
GROUP BY institution
ORDER BY average_s

# Report: Experimenting with NL2SQL Prompts

## Objective
The goal of this experiment was to evaluate the effectiveness of different prompt styles for generating SQL queries using a language model. We tested three variations of prompts: a baseline prompt (`t0_prompt`), a concise prompt (`simple_prompt`), and a conversational prompt (`t1_prompt`). The prompts were evaluated on their ability to generate accurate and efficient SQL queries for three predefined user queries.

## Queries Tested
1. "highest paid employee's name"
2. "institution whose graduates have the highest average salary"
3. "employee with the lowest salary and their institution"

## Results and Observations

### Baseline Prompt (`t0_prompt`)
- **Strengths**: The baseline prompt provided detailed context about the database schema and explicitly instructed the model to generate valid SQL queries.
- **Weaknesses**: While the SQL queries generated were mostly accurate, the prompt was verbose, which may have introduced unnecessary complexity. This verbosity could potentially lead to slower response times.

### Concise Prompt (`simple_prompt`)
- **Strengths**: The concise prompt was minimalistic, providing only the essential details about the database schema. It performed well for straightforward queries like "highest paid employee's name."
- **Weaknesses**: For more complex queries, such as "institution whose graduates have the highest average salary," the concise prompt occasionally produced incomplete or incorrect SQL queries. This suggests that the lack of detailed instructions may have limited the model's ability to handle nuanced requests.

### Conversational Prompt (`t1_prompt`)
- **Strengths**: The conversational prompt framed the task in a more natural and contextual manner, which seemed to improve the model's understanding of complex queries. It performed well across all tested queries, generating efficient and accurate SQL.
- **Weaknesses**: While effective, the conversational style may not be suitable for all use cases, particularly those requiring strict adherence to technical language.

## Key Findings
1. **Prompt Style Matters**: The style and level of detail in the prompt significantly impact the quality of the generated SQL queries.
2. **Complexity vs. Simplicity**: While concise prompts are efficient for simple queries, they may struggle with more complex tasks. Conversely, detailed prompts provide better guidance but may introduce unnecessary complexity.
3. **Conversational Prompts Are Promising**: Framing the task in a conversational manner can improve the model's performance, especially for complex queries.

## Lessons Learned
- Providing a clear and structured description of the database schema is crucial for generating accurate SQL queries.
- Including examples or framing the task in a conversational context can enhance the model's understanding and output quality.
- Striking a balance between verbosity and conciseness is key to designing effective prompts.

## Recommendations
- Use detailed prompts for complex queries where accuracy is critical.
- Opt for concise prompts for simple queries to improve efficiency.
- Experiment with conversational prompts for tasks requiring nuanced understanding.